I will just write on each line a function to be called which we should try a net on. We could try combining functions later.

In [1]:
import scipy
from sklearn.preprocessing import StandardScaler

Functions 1:Standard MFCC

In [3]:
def mfcc_(song, n=20):
    return librosa.feature.mfcc(y = song, n_mfcc = n)


Function 2: MFCC without low

In [ ]:
def mfcc_cut(song, n=20, cutoff = 800):
    #remove bottom 800 hz of frequencies
    return librosa.feature.mfcc(y = song, fmin = cutoff, n_mfcc = n)

Function 3: LFCC with low

In [4]:
def LFCC(song, n=20, n_fft=2048, hop_length=512):
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    DB = librosa.amplitude_to_db(D, ref=np.max)
    
    time = DB.shape[1]
    ans=np.zeros((n,time), dtype=float, order='C')
    for x in range(0,time):
        col = DB[:,x]
        ans[:,x] = (scipy.fft.dct(col)[0:n])
    return ans

There might be a faster implementation of the above by replacing the for loop over time with multiplying by a matrix. Might be because I don't know how optimized scipy is at multiplying non square matrices.

Function 4:LFCC without low

In [5]:
def LFCC_no_low(song, n=20, cutoff = 800, n_fft=2048, hop_length=512):
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    cut = int(float(cutoff)/(librosa.fft_frequencies()[1]))
    D = D[cut:]
    #remove bottom 10.7*cutoff frequencies. The number 10.7 is the value of librosa.fft_frequencies()[1]
    DB = librosa.amplitude_to_db(D, ref=np.max)
    
    time = DB.shape[1]
    ans=np.zeros((n,time), dtype=float, order='C')
    for x in range(0,time):
        col = DB[:,x]
        ans[:,x] = (scipy.fft.dct(col)[0:n])
    return ans

Again maybe we should use a faster implementation as in function 2

Function 5:LFCC without low cleaned

In [7]:
def LFCC_no_low_clean(song, n=20, cutoff = 800, n_fft=2048, hop_length=512):
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    cut = int(float(cutoff)/(librosa.fft_frequencies()[1]))
    D = D[cut:]
    DB = librosa.amplitude_to_db(D, ref=np.max)
    for x in range(0,DB.shape[0]):
        med = np.median(DB[x, :])
        DB[x, :] = DB[x, :]-med
        
    time = DB.shape[1]
    ans=np.zeros((n,time), dtype=float, order='C')
    for x in range(0,time):
        col = DB[:,x]
        ans[:,x] = (scipy.fft.dct(col)[0:n])
    return ans

Again implementation

Function 6:MFCC without low and cleaned

In [ ]:
def mfcc_cut_cleaned(song, n=20, cutoff = 800, n_fft=2048, hop_length=512):
    Dmel = librosa.feature.melspectrogram(y=song, fmin = cutoff, n_fft=n_fft,  hop_length=hop_length)
    DBmel=librosa.power_to_db(Dmel)
    for x in range(0,DBmel.shape[0]):
        med = np.median(DBmel[x, :])
        DBmel[x, :] = DBmel[x, :]-med
    return librosa.feature.mfcc(S=DBmel, n_mfcc = n)

### Function 7: Main Frequency via variance and mean

In [ ]:
def main_freq(song, cutoff = 800):
    D = np.abs(librosa.stft(song))
    cut = int(float(cutoff)/(librosa.fft_frequencies()[1]))
    D = D[cut:]
    DB = librosa.amplitude_to_db(D, ref=np.max)
    maxmean = np.mean(DB[0,:])
    main_meanfreq = 0
    maxvar = np.var(DB[0,:])
    main_varfreq = 0
    for x in range(1,DB.shape[0]):
        mean = np.mean(DB[x,:])
        var = np.var(DB[x,:])
        if mean > maxmean:
            maxmean = mean
            main_meanfreq = x
        if var > maxvar:
            maxvar = var
            main_varfreq = x
    return main_varfreq * librosa.fft_frequencies()[1], main_meanfreq * librosa.fft_frequencies()[1]

### Function to pull the avg and var mfccs from the loudest half second of the recroding
song = librosa-loaded song,
sr = sampling rate from song,
n = number of mfccs


In [26]:
def get_hs_avg_var_mfccs(song, sr=22050, n = 40, cutoff = 800):
    bound = round(sr/4) #sr/4 gives the half second interval, sr/8 does better in limitted testing
    max_ampj = np.argmax(abs(song))
    time_start, time_stop = max_ampj - bound, max_ampj + bound
    if time_start < 0:
        time_start, time_stop = 0, 2*bound
    if time_stop > len(song):
        time_start, time_stop = len(song) - 2*bound, len(song)
    mfccs = librosa.feature.mfcc(y = song[time_start:time_stop], fmin = cutoff, n_mfcc = n)
    mfcc_avg = np.mean(mfccs, axis = 1)
    mfcc_var = np.var(mfccs, axis = 1)
    if time_start < 0: return (np.full(20, np.nan), np.full(20, np.nan))
    else: 
        return (mfcc_avg, mfcc_var)


### Function to pull all the mfccs from the loudest half second (average over small time samples?)

In [15]:
def get_hs_mfccs(song, sr=22050, n = 40, cutoff = 800):
    bound = round(sr/8) #sr/4 gives the half second interval
    max_ampj = np.argmax(abs(song))
    time_start, time_stop = max_ampj - bound, max_ampj + bound
    if time_start < 0:
        time_start, time_stop = 0, 2*bound
    if time_stop > len(song):
        time_start, time_stop = len(song) - 2*bound, len(song)
    return librosa.feature.mfcc(y = song[time_start:time_stop], fmin = cutoff, n_mfcc = n)

### Function to get periods. Returns an array of length 36 where the i-th entry being large says that probably has some periodic feature with between 10i and 11i periods per minute

In [6]:
#should rename to chirp frequencies instead of periods
def get_periods(song, cutoff = 800, tsps = 44):
    #tsps is time stamps per second
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    cut = int(float(cutoff)/(librosa.fft_frequencies()[1]))
    D = D[cut:]

    #perform fft on the volume of the noise
    data = D.sum(axis=0)
    time =  D.shape[1]
    halftime = int(time/2)
    data2 = data- data.mean()
    fft = np.absolute(scipy.fft.fft(data2))
    fft2 = fft[2:halftime]
    scaler = StandardScaler()
    scaled_fft = scaler.fit_transform((fft2.T).reshape(-1,1))

    #bucket together close by periods to make data smaller. I'm kinda arbitrarily squaring to make more prevalent periods noticed
    buckets = np.zeros((48))
    counter = np.zeros((48))
    mini = min(data.shape[0],int(8*time/tsps -3))
    for i in range(0,mini):
        #44 is the number of time samples per second
        bucket_num = int((i+2)/time*tsps*60/10)
        if scaled_fft[i]>0:
            buckets[bucket_num] = buckets[bucket_num] + np.square(scaled_fft[i,0])
        counter[bucket_num] = counter[bucket_num]+1
    for j in range(0,48):
        if buckets[j]>0:
            buckets[j] = buckets[j]/counter[j]

    return buckets